# Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import re

# Загружаем и смотрим данные

In [2]:
data_raw = pd.read_csv('all_auto_ru_09_09_2020.csv.zip')

In [3]:
data_raw.head(2).T

,0,1
bodyType,Седан,Седан
brand,AUDI,AUDI
color,040001,EE1D19
fuelType,бензин,бензин
modelDate,1990.0,1982.0
name,2.8 MT (174 л.с.) 4WD,1.8 MT (90 л.с.)
numberOfDoors,4.0,4.0
productionDate,1991,1986
vehicleConfiguration,SEDAN MECHANICAL 2.8,SEDAN MECHANICAL 1.8
vehicleTransmission,MECHANICAL,MECHANICAL


In [4]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89378 entries, 0 to 89377
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              89377 non-null  object 
 1   brand                 89378 non-null  object 
 2   color                 89378 non-null  object 
 3   fuelType              89378 non-null  object 
 4   modelDate             89377 non-null  float64
 5   name                  89377 non-null  object 
 6   numberOfDoors         89377 non-null  float64
 7   productionDate        89378 non-null  int64  
 8   vehicleConfiguration  89377 non-null  object 
 9   vehicleTransmission   89377 non-null  object 
 10  engineDisplacement    89377 non-null  object 
 11  enginePower           89377 non-null  float64
 12  description           86124 non-null  object 
 13  mileage               89378 non-null  int64  
 14  Комплектация          89378 non-null  object 
 15  Привод             

In [5]:
data_raw_profile = ProfileReport(data_raw, title="Pandas Profiling Report")
# data_raw_profile.to_notebook_iframe()

Подход такой же, как и при обработке датасета соревнования.

Что надо сделать:
- оставить данные, только есть цена
- сравнить категориальные признаки с датасетом соревнования, привести к такому же типу
- названия признаков привести к одному виду
- удалить дубликаты

In [6]:
# удалим бесценное, сильно пустое и дубликаты
data_raw = data_raw.dropna(axis=0, subset=['price']).dropna(axis=0, thresh=20).drop_duplicates()

# Предобработка v2

## Копируем основу

In [7]:
# удалим бесполезное
data_1 = data_raw.drop(columns=['Состояние', 'Таможня', 'Владение', 'hidden'])

In [8]:
# удалим требующее обработки, добавим потом из data_raw
data_1.drop(columns=['ПТС', 'Руль', 'Комплектация', 'name', 'model', 
                     'start_date', 'engineDisplacement', 'vehicleConfiguration'], inplace=True)
# оставим description для CatBoost
# model пойдёт в сборный признак

## Правим названия

In [9]:
# приведем названия к единому стилю
rule_rename = {'bodyType': 'body_type',
               'fuelType': 'fuel_type',
               'modelDate': 'model_year',
               'model': 'model_name',
               'numberOfDoors': 'n_doors',
               'productionDate': 'production_year',
               'vehicleConfiguration': 'vehicle_configuration',
               'vehicleTransmission': 'vehicle_transmission',
               'engineDisplacement': 'engine_displacement',
               'enginePower': 'engine_power',
               'Владельцы': 'n_owners',
               'Привод': 'drive_type'}
data_1.rename(columns=rule_rename, inplace=True)

## Простые обработки признаков

In [10]:
# ПТС - это бинарный признак, оригинал или дубликат
data_1['is_original_techpass'] = data_raw['ПТС'].apply(lambda x: 1 if x in ['ORIGINAL', 'Оригинал'] else 0).astype('int8')

In [11]:
# Руль - это бинарный признак, левый или правый
data_1['is_lefthand_drive'] = data_raw['Руль'].apply(lambda x: 1 if x in ['LEFT', 'Левый'] else 0).astype('int8')

In [12]:
# сделаем объем двигателя числом
engine_displacement_re = data_raw['name'].apply(lambda x : re.match('\d\.\d', x))
data_1['engine_displacement'] = engine_displacement_re.apply(lambda x: float(x.group(0)) if x is not None else np.NaN)

In [13]:
# укрупним категории типа кузова
# теперь пусть роадстер будет отдельным типом
def generic_body_type(raw_body_type):
    """
    Функция ищет ключевые слова в описании кузова и возвращает тип кузова
    """
    result = raw_body_type
    body_type = raw_body_type.lower()
    generic_dict = {'лифтбек': 'лифтбек',
                    'фастбек': 'лифтбек',
                    'седан': 'седан',
                    'внедорожник': 'внедорожник',
                    'хэтчбек': 'хэтчбек',
                    'универсал': 'универсал',
                    'пикап': 'пикап',
                    'купе': 'купе',
                    'кабриолет': 'кабриолет',
                    'родстер': 'родстер',
                    'тарга': 'родстер',
                    'вэн': 'минивэн',
                    'лимузин': 'лимузин',
                    'фургон': 'фургон'}
    for keyword in generic_dict.keys():
        if body_type.find(keyword) >= 0:
            result = generic_dict[keyword]
            break
    return result

data_1['body_type'] = data_1['body_type'].fillna(value='unknown').apply(generic_body_type)
data_1['body_type'].unique()

array(['седан', 'универсал', 'хэтчбек', 'кабриолет', 'купе', 'лифтбек',
       'внедорожник', 'минивэн', 'родстер', 'пикап', 'фургон', 'лимузин'],
      dtype=object)

In [14]:
# назовём цвет кузова
# https://hysy.org/colors/
def generic_color(raw_color):
    """
    Функция проверяет цвет и даёт ему название
    """
    result = raw_color
    generic_dict = {'040001': 'чёрный',
                    'EE1D19': 'красный',
                    '0000CC': 'синий',
                    'CACECB': 'серый',
                    '007F00': 'зелёный',
                    'FAFBFB': 'белый',
                    '97948F': 'серый',
                    '22A0F8': 'синий',
                    '660099': 'фиолетовый',
                    '200204': 'коричневый',
                    'C49648': 'бежевый',
                    'DEA522': 'жёлтый',
                    '4A2197': 'фиолетовый',
                    'FFD600': 'жёлтый',
                    'FF8649': 'розовый',
                    'FFC0CB': 'розовый',
                    'пурпурный': 'фиолетовый',
                    'голубой': 'синий',
                    'золотистый': 'жёлтый',
                    'серебристый': 'серый',
                    'оранжевый': 'жёлтый'}
    for keyword in generic_dict.keys():
        if raw_color.find(keyword) >= 0:
            result = generic_dict[keyword]
            break
    return result

data_1['color'] = data_1['color'].fillna(value='unknown').apply(generic_color)
data_1['color'].unique()

array(['чёрный', 'красный', 'синий', 'серый', 'зелёный', 'белый',
       'фиолетовый', 'коричневый', 'бежевый', 'жёлтый', 'розовый'],
      dtype=object)

In [15]:
data_1['model_year'] = data_1['model_year'].astype('int')

In [16]:
data_1['n_doors'] = data_1['n_doors'].astype('int')

In [17]:
# назовём тип КПП
def generic_transmission(raw_gearbox):
    """
    Функция даёт название КПП
    """
    result = raw_gearbox
    generic_dict = {'MECHANICAL': 'механика',
                    'AUTOMATIC': 'автомат',
                    'ROBOT': 'робот',
                    'VARIATOR': 'вариатор',
                    'роботизированная': 'робот',
                    'механическая': 'механика',
                    'автоматическая': 'автомат',
                    'вариатор': 'вариатор'
                   }
    for keyword in generic_dict.keys():
        if raw_gearbox.find(keyword) >= 0:
            result = generic_dict[keyword]
            break
    return result

data_1['vehicle_transmission'] = data_1['vehicle_transmission'].fillna(value='unknown').apply(generic_transmission)
data_1['vehicle_transmission'].unique()

array(['механика', 'автомат', 'робот', 'вариатор'], dtype=object)

In [18]:
# назовём количество владельцев
def generic_owners(raw_owners):
    """
    Функция даёт название КПП
    """
    result = str(raw_owners)
    generic_dict = {'1': '1',
                    '2': '2',
                    '3': '3+',
                   }
    for keyword in generic_dict.keys():
        if str(raw_owners).find(keyword) >= 0:
            result = generic_dict[keyword]
            break
    return result

data_1['n_owners'] = data_1['n_owners'].fillna(value='0').apply(generic_owners)
data_1['n_owners'].unique()

array(['3+', '1', '2', '0'], dtype=object)

In [19]:
# посмотрим на этих неизвестных владельев
# display(data_1[data_1['n_owners'] == 'unknown'].sample(5))
# display(data_1[data_1['n_owners'] == 'unknown']['mileage'].unique())
# это новые автомобили, значит отметим выше, что у них 0 владельцев

## Более сложные признаки

In [20]:
# соберём марку и модель в один признак
data_1['full_model_name'] = (data_raw['brand'] + ' ' + data_raw['model']).str.lower()
data_1['full_model_name'].unique()

array(['audi 100', 'audi 200', 'audi 80', ..., 'ssang_yong rexton',
       'ssang_yong rodius', 'ssang_yong stavic'], dtype=object)

# Посмотрим на результат предобработки

In [21]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86853 entries, 0 to 89377
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   body_type             86853 non-null  object 
 1   brand                 86853 non-null  object 
 2   color                 86853 non-null  object 
 3   fuel_type             86853 non-null  object 
 4   model_year            86853 non-null  int32  
 5   n_doors               86853 non-null  int32  
 6   production_year       86853 non-null  int64  
 7   vehicle_transmission  86853 non-null  object 
 8   engine_power          86853 non-null  float64
 9   description           83640 non-null  object 
 10  mileage               86853 non-null  int64  
 11  drive_type            86853 non-null  object 
 12  n_owners              86853 non-null  object 
 13  price                 86853 non-null  float64
 14  is_original_techpass  86853 non-null  int8   
 15  is_lefthand_drive  

In [22]:
data_1_profile = ProfileReport(data_1, title="Data_EDA_v1 Profiling Report")
# data_1_profile.to_notebook_iframe()

# Сохраним результат предобработки

In [23]:
compression_opts = dict(method='zip', archive_name='EDAv2_Train.csv')  
data_1.to_csv('EDAv2_Train.zip', index=False, compression=compression_opts)

# Посмотрим внимательно на цены

In [27]:
data_1['price'].sample(10)

In [29]:
# гипотеза - все цены кратны 1000 рублей
price = data_1['price']

In [34]:
(((price/1000).astype('int')*1000 - price) != 0).sum()
# довольно мало ~10% не кратны 1000

8099

In [35]:
price[((price/1000).astype('int')*1000 - price) != 0]
# при этом даже не кратные, очень близки

40       109999.0
110      199999.0
154       87500.0
205       99999.0
231       89999.0
           ...   
88868    549999.0
89013    529900.0
89019    534900.0
89041    749750.0
89202    429999.0
Name: price, Length: 8099, dtype: float64